In [5]:
# 1. 알고(구현): 백준 2161 (카드1) - https://www.acmicpc.net/problem/2161
import sys
from collections import deque

from io import StringIO
sys.stdin = StringIO('''7''')

input = sys.stdin.readline


def solve():

    N = int(input())
    dq = deque(range(1, N + 1))
    answer = []

    while dq:
        answer.append(dq.popleft())
        dq.rotate(-1)

    print(*answer)

solve()

1 3 5 7 4 2 6


In [ ]:
# 2. 알고(수학): 백준 2292 (벌집) - https://www.acmicpc.net/problem/2292
import sys

from io import StringIO
sys.stdin = StringIO('''13''')

input = sys.stdin.readline


def solve():
    # ak = 1 + sigma(i,k-1)6i = 1 + 3k(k-1)
    N = int(input())

    layer = 1
    while True:
        max_num = 1 + 3*layer*(layer - 1)
        if N <= max_num:
            break
        layer += 1
    print(layer)


solve()

3


In [45]:
# 3. 알고(Map): 백준 17219 (비밀번호 찾기) - https://www.acmicpc.net/problem/17219
import sys

from io import StringIO
sys.stdin = StringIO('''16 4
noj.am IU
acmicpc.net UAENA
startlink.io THEKINGOD
google.com ZEZE
nate.com VOICEMAIL
naver.com REDQUEEN
daum.net MODERNTIMES
utube.com BLACKOUT
zum.com LASTFANTASY
dreamwiz.com RAINDROP
hanyang.ac.kr SOMEDAY
dhlottery.co.kr BOO
duksoo.hs.kr HAVANA
hanyang-u.ms.kr OBLIVIATE
yd.es.kr LOVEATTACK
mcc.hanyang.ac.kr ADREAMER
startlink.io
acmicpc.net
noj.am
mcc.hanyang.ac.kr''')

input = sys.stdin.readline


def solve():

    N, M = map(int, input().split())

    passwords = dict()
    for _ in range(N):
        url, pswd = input().split()
        passwords[url] = pswd
    
    result = []
    for _ in range(M):
        url = input().rstrip()
        result.append(passwords[url])

    print("\n".join(result))

solve()


THEKINGOD
UAENA
IU
ADREAMER


In [54]:
# 4. 알고(그리디): 백준 11508 (2+1 세일) - https://www.acmicpc.net/problem/11508
import sys

from io import StringIO
sys.stdin = StringIO('''6
6
4
5
5
5
5''')

input = sys.stdin.readline


def solve():

    N = int(input())

    prices = [int(input()) for _ in range(N)]
    prices.sort(reverse=True)

    print(sum(prices) - sum(prices[2::3]))


solve()

21


```sql
--5. SQL(Lv.1): 흉부외과 또는 일반외과 의사 목록 출력하기 
-- https://school.programmers.co.kr/learn/courses/30/lessons/132203
SELECT
    DR_NAME,
    DR_ID,
    MCDP_CD,
    DATE_FORMAT(HIRE_YMD, '%Y-%m-%d') AS HIRE_YMD
FROM 
    DOCTOR
WHERE
    MCDP_CD IN ('CS', 'GS')
ORDER BY
    HIRE_YMD DESC,
    DR_NAME ASC;
```

In [ ]:
# 1. 알고(시뮬): 백준 16234 (인구 이동) - https://www.acmicpc.net/problem/16234
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 2. 알고(DP): 백준 1520 (내리막 길) - https://www.acmicpc.net/problem/1520
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 3. 알고(그래프): 백준 11779 (최소비용 구하기 2) - https://www.acmicpc.net/problem/11779
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 4. 알고(백트래킹): 백준 1062 (가르침) - https://www.acmicpc.net/problem/1062
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


```sql
-- 5. SQL(Lv.4): 주문량이 많은 아이스크림들 조회하기 
-- https://school.programmers.co.kr/learn/courses/30/lessons/133027
```